In [1]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from utils.Common import Config

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [3]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [4]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [5]:
# pass feature to pipeline and convert it to numerical data
X = DataPipeline(Config.num_attribs,Config.cat_attribs).process(X)

In [6]:
X.isna().sum().sum()

0

In [7]:
Y.value_counts().tolist()

[14246, 2201]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y, random_state=42)

In [9]:
smote_minority = SMOTE(sampling_strategy = "minority", random_state=42)
X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


In [10]:
# Create param grid
param_grid = [
    {
    'penalty' : ['l2'],
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver' : ['lbfgs', 'newton-cg','sag', 'saga' ]
    }
]


clf = GridSearchCV(estimator=LogisticRegression(random_state=42),param_grid=param_grid,n_jobs=10,cv=10)
best_clf = clf.fit(X_train_sm, y_train_sm)
best_clf.best_estimator_
best_clf.best_score_

0.6946297814523146

In [11]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.6796352583586626

In [15]:
best_model.score(X_train, y_train)

0.6867066960553317

In [12]:
from sklearn.metrics import classification_report

y_train_pred = best_model.predict(X_train)
y_train_sm_pred = best_model.predict(X_train_sm)
y_test_pred = best_model.predict(X_test)

print(classification_report(y_train,y_train_pred))
print(classification_report(y_train_sm,y_train_sm_pred))
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.94      0.69      0.79     11396
           1       0.25      0.69      0.37      1761

    accuracy                           0.69     13157
   macro avg       0.59      0.69      0.58     13157
weighted avg       0.84      0.69      0.74     13157

              precision    recall  f1-score   support

           0       0.70      0.69      0.69     11396
           1       0.69      0.71      0.70     11396

    accuracy                           0.70     22792
   macro avg       0.70      0.70      0.70     22792
weighted avg       0.70      0.70      0.70     22792

              precision    recall  f1-score   support

           0       0.93      0.68      0.79      2850
           1       0.24      0.65      0.35       440

    accuracy                           0.68      3290
   macro avg       0.58      0.67      0.57      3290
weighted avg       0.84      0.68      0.73      3290



In [13]:
best_clf.best_params_
# {'C': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}


{'C': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}

In [14]:
import joblib
best_model = LogisticRegression(C= 1000, penalty='l2', solver= 'newton-cg')
best_model.fit(X_train_sm, y_train_sm)
joblib.dump(best_model,'../models/best_model_logistic_regression.pkl')


['../models/best_model_logistic_regression.pkl']